In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Load Dataset
df = pd.read_csv("../input/final-data/final_data.csv")

In [ ]:
df = df.drop("Doc_ID", axis=1)

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.isna().sum()

In [ ]:
df.dtypes

In [ ]:
#Encoding for Categorical Values
data2 = pd.get_dummies(df, columns=["Line_Length"])
data2 = pd.get_dummies(data2, columns=["Line_Words"])
data2 = pd.get_dummies(data2, columns=["Inline_space_tab"])
data2 = pd.get_dummies(data2, columns=["Indent_space_tab"])
data2 = pd.get_dummies(data2, columns=["identifiers_with_underscore"])
data2 = pd.get_dummies(data2, columns=["Number_of_Identifiers"])
data2 = pd.get_dummies(data2, columns=["Number_of_keywords"])

In [ ]:
#Scaling Numerical columns
from sklearn.preprocessing import StandardScaler
num_cols = ['Number_of_Lines', 'number_of_spaces', 'number_of_underscores']
std = StandardScaler()
scaled = std.fit_transform(df[num_cols])
scaled = pd.DataFrame(scaled,columns=num_cols)
data2.drop(columns = num_cols,axis = 1, inplace= True)
data2 = data2.merge(scaled,left_index=True,right_index=True,how = "left")

In [ ]:
data2

In [ ]:
data2.isna().sum()

In [ ]:
data2.dropna()

In [ ]:
train = data2[0:252]

In [ ]:
train

In [ ]:
Y = train.Programmer
X = train.drop("Programmer", axis=1)

In [ ]:
#K-Fold CV for splitting up data
from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold(n_splits=4,shuffle=True,random_state=None)
pred_test_full =0
cv_score =[]
i=1
for train_index,test_index in kf.split(X,Y):
    print('{} of KFold {}'.format(i,kf.n_splits))
    X_train,X_test = X.loc[train_index],X.loc[test_index]
    Y_train,Y_test = Y.loc[train_index],Y.loc[test_index]

In [ ]:
#K-Nearest Neighbour Classifier
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier()
model.fit(X_train, Y_train)

In [ ]:
Y_pred = model.predict(X_test)
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report 
print (classification_report(Y_test, Y_pred))
cm = confusion_matrix(Y_test, Y_pred) #Confusion Matrix
print(cm)
accuracy_score(Y_test, Y_pred) #Accuracy

In [ ]:
print('Accuracy of K-NN classifier on training set: {:.2f}'
     .format(model.score(X_train, Y_train)))
print('Accuracy of K-NN classifier on test set: {:.2f}'
     .format(model.score(X_test, Y_test)))

In [ ]:
Y_test.value_counts()
print('True', Y_test.values[0:100])
print('Pred', Y_pred[0:100])

In [ ]:
y_test = pd.get_dummies(Y_test, columns=["Programmer"])
y_pred = pd.get_dummies(Y_pred, columns=["Programmer"])

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn import metrics
probabilities = model.predict_proba(X_test)
model_roc_auc = roc_auc_score(y_test, y_pred) #AUC Value
print ("Area under curve : ",model_roc_auc,"\n")

In [ ]:
test

In [ ]:
test = data2[252:]
X_testing = test.drop("Programmer", axis=1)
y_sub = model.predict(X_testing)
#X_testing
y_sub

In [ ]:
#Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
rfor = RandomForestClassifier()
rfor.fit(X_train, Y_train)

In [ ]:
Y_pred = rfor.predict(X_test)
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report 
print (classification_report(Y_test, Y_pred)) #Confusion Matrix
cm = confusion_matrix(Y_test, Y_pred)
print(cm)
accuracy_score(Y_test, Y_pred) #Accuracy

In [ ]:
y_test = pd.get_dummies(Y_test, columns=["Programmer"])
y_pred = pd.get_dummies(Y_pred, columns=["Programmer"])
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn import metrics
probabilities = rfor.predict_proba(X_test)
model_roc_auc = roc_auc_score(y_test, y_pred) #AUC Value
print ("Area under curve : ",model_roc_auc,"\n")

In [ ]:
#Multi-Layer Perceptron Classifier
from sklearn.neural_network import MLPClassifier
mlpc = MLPClassifier(random_state=1, max_iter=1000)
mlpc.fit(X_train, Y_train)

In [ ]:
Y_pred = mlpc.predict(X_test)
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report 
print (classification_report(Y_test, Y_pred))
cm = confusion_matrix(Y_test, Y_pred) #Confusion Matrix
print(cm)
accuracy_score(Y_test, Y_pred) #Accuracy

In [ ]:
print('Accuracy of MLP classifier on training set: {:.2f}'
     .format(mlpc.score(X_train, Y_train)))
print('Accuracy MLP classifier on test set: {:.2f}'
     .format(mlpc.score(X_test, Y_test)))

In [ ]:
y_test = pd.get_dummies(Y_test, columns=["Programmer"])
y_pred = pd.get_dummies(Y_pred, columns=["Programmer"])
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn import metrics
probabilities = mlpc.predict_proba(X_test)
model_roc_auc = roc_auc_score(y_test, y_pred) #AUC value
print ("Area under curve : ",model_roc_auc,"\n")

In [ ]:
test = data2[252:]
X_testing = test.drop("Programmer", axis=1)
y_sub = mlpc.predict(X_testing)
#X_testing
y_sub

In [ ]:
#Linear Discriminant Analysis Classifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, Y_train)

In [ ]:
Y_pred = lda.predict(X_test)
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report 
print (classification_report(Y_test, Y_pred))
cm = confusion_matrix(Y_test, Y_pred) #Confusion Matrix
print(cm)
accuracy_score(Y_test, Y_pred) #Accuracy

In [ ]:
print('Accuracy of LDA classifier on training set: {:.2f}'
     .format(lda.score(X_train, Y_train)))
print('Accuracy of LDA classifier on test set: {:.2f}'
     .format(lda.score(X_test, Y_test)))

In [ ]:
y_test = pd.get_dummies(Y_test, columns=["Programmer"])
y_pred = pd.get_dummies(Y_pred, columns=["Programmer"])
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn import metrics
probabilities = lda.predict_proba(X_test)
model_roc_auc = roc_auc_score(y_test, y_pred) #AUC Value
print ("Area under curve : ",model_roc_auc,"\n")

In [ ]:
#Stochastic Gradient Descent Classifier
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
clf = make_pipeline(StandardScaler(),SGDClassifier(max_iter=1000, tol=1e-3))
clf.fit(X_train, Y_train)

In [ ]:
Y_pred = clf.predict(X_test)
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report 
print (classification_report(Y_test, Y_pred))
cm = confusion_matrix(Y_test, Y_pred) #Confusion Matrix
print(cm)
accuracy_score(Y_test, Y_pred) #Accuracy

In [ ]:
print('Accuracy of SGD classifier on training set: {:.2f}'
     .format(clf.score(X_train, Y_train)))
print('Accuracy of SGD classifier on test set: {:.2f}'
     .format(clf.score(X_test, Y_test)))

In [ ]:
y_test = pd.get_dummies(Y_test, columns=["Programmer"])
y_pred = pd.get_dummies(Y_pred, columns=["Programmer"])
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn import metrics
probabilities = clf.predict_proba(X_test)
model_roc_auc = roc_auc_score(y_test, y_pred) 
print ("Area under curve : ",model_roc_auc,"\n") #AUC Value

In [ ]:
#Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier().fit(X_train, Y_train)

In [ ]:
Y_pred = dtc.predict(X_test)
#from sklearn.metrics import mean_absolute_error
#mae = mean_absolute_error(Y_test, Y_pred)
#mae
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report 
print (classification_report(Y_test, Y_pred))
cm = confusion_matrix(Y_test, Y_pred) #Confuion Matrix
print(cm)
accuracy_score(Y_test, Y_pred) #Accuracy

In [ ]:
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(dtc.score(X_train, Y_train)))
print('Accuracy of Decision Tree classifier on test set: {:.2f}'
     .format(dtc.score(X_test, Y_test)))

In [ ]:
y_test = pd.get_dummies(Y_test, columns=["Programmer"])
y_pred = pd.get_dummies(Y_pred, columns=["Programmer"])
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn import metrics
probabilities = dtc.predict_proba(X_test)
model_roc_auc = roc_auc_score(y_test, y_pred) 
print ("Area under curve : ",model_roc_auc,"\n") #AUC Value

In [ ]:
import matplotlib.pyplot as plt
plt.bar(['KNN', 'Random Forest', 'MLPC', 'LDA', 'SGDC', 'Decision Tree'], [0.58, 0.79, 0.78, 0.63, 0.86, 0.69])
plt.bar(['KNN', 'Random Forest', 'MLPC', 'LDA', 'SGDC', 'Decision Tree'], [0.58, 0.75, 0.75, 0.48, 0.82, 0.58])
plt.bar(['KNN', 'Random Forest', 'MLPC', 'LDA', 'SGDC', 'Decision Tree'], [0.57, 0.74, 0.73, 0.51, 0.82, 0.58])